In [3]:
from pycocotools.coco import COCO
import numpy as np
import cv2

# Create Mask Data

In [5]:
def get_classname(classID, cats):
    for i in range(len(cats)):
        if cats[i]['id']==classID:
            return cats[i]['name']
    return "None"


data = 'train' # or 'val'

dataset_path  = '/opt/ml/segmentation/input/data'
new_dataset_path = f'/opt/ml/segmentation/moon/dataset/annotations/{data}'
anns_file_path = dataset_path + '/' + f'{data}.json'

coco = COCO(anns_file_path)
image_id = coco.getImgIds()
image_infos = coco.loadImgs(image_id)
category_names = ['Backgroud',
 'General trash',
 'Paper',
 'Paper pack',
 'Metal',
 'Glass',
 'Plastic',
 'Styrofoam',
 'Plastic bag',
 'Battery',
 'Clothing']

loading annotations into memory...
Done (t=5.00s)
creating index...
index created!


In [6]:
image_num = 0
for image_info in image_infos:
    ann_ids = coco.getAnnIds(imgIds=image_info['id'])
    anns = coco.loadAnns(ann_ids)

    # Load the categories in a variable
    cat_ids = coco.getCatIds()
    cats = coco.loadCats(cat_ids)

    # masks : size가 (height x width)인 2D
    # 각각의 pixel 값에는 "category id" 할당
    # Background = 0
    masks = np.zeros((image_info["height"], image_info["width"]))
    # General trash = 1, ... , Cigarette = 10
    anns = sorted(anns, key=lambda idx : len(idx['segmentation'][0]), reverse=False)
    for i in range(len(anns)):
        className = get_classname(anns[i]['category_id'], cats)
        pixel_value = category_names.index(className)
        masks[coco.annToMask(anns[i]) == 1] = pixel_value
    masks = masks.astype(np.int8)

    file_name = os.path.join(new_dataset_path,f"{image_num:04d}.png")
    image_num +=1
    cv2.imwrite(file_name,masks)

# Move train Dataset

In [1]:
import os
import shutil
from pycocotools.coco import COCO

In [4]:
dataset_path  = '/opt/ml/segmentation/input/data'
anns_file_path = dataset_path + '/' + 'train.json'
new_dataset_path = '/opt/ml/segmentation/moon/dataset/images/train'


if not os.path.exists(new_dataset_path):
    os.mkdir(new_dataset_path)

coco = COCO(anns_file_path)
image_id = coco.getImgIds()
image_infos = coco.loadImgs(image_id)

image_num = 0

for image_info in image_infos:
    src_file = os.path.join(dataset_path,image_info['file_name'])
    dest_file = os.path.join(new_dataset_path,f"{image_num:04d}.jpg")
    image_num+=1
    shutil.copy(src_file,dest_file)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


# Move Test dataset

In [5]:
import os
import shutil
from pycocotools.coco import COCO

In [21]:
dataset_path  = '/opt/ml/segmentation/input/data'
anns_file_path = dataset_path + '/' + 'test.json'
new_dataset_path = '/opt/ml/segmentation/moon/dataset/images/test'


if not os.path.exists(new_dataset_path):
    os.mkdir(new_dataset_path)

coco = COCO(anns_file_path)
image_id = coco.getImgIds()
image_infos = coco.loadImgs(image_id)


for image_info in image_infos:
    src_file = os.path.join(dataset_path,image_info['file_name'])
    dest_file = os.path.join(new_dataset_path,'+'.join(image_info['file_name'].split('/')))
    shutil.copy(src_file,dest_file)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [20]:
'+'.join(image_info['file_name'].split('/'))

'batch_01_vt+0021.jpg'